In [1]:
%matplotlib inline
import numpy as np
import ephem
import matplotlib
import matplotlib.pyplot as plt

In [2]:
def get_twilight(obs):
    '''
    Compute the twilight times for an observer.
    '''
    sun = ephem.Sun()

    start_night = obs.next_setting(sun)
    end_night = obs.next_rising(sun)

    if start_night > end_night:
        start_night = obs.date

    return start_night, end_night

In [3]:
def alt2airmass(alt):
    '''
    Calculate the airmass at the given altitude (in radians).

    Uses the Hardie (1962) interpolation function.
    '''

    if isinstance(alt, list):
        alt[alt < 0.07] = 0.07
    else:
        alt = max(alt, 0.07)

    secz = 1.0 / np.sin(alt)
    seczm1 = secz - 1.0

    airm = (
        secz
        - 0.0018167*seczm1
        - 0.002875*seczm1**2
        - 0.0008083*seczm1**3
    )

    return airm

In [4]:
def radec2airmass(obs, ra, dec):
    b = ephem.FixedBody()
    b._ra = np.radians(ra)
    b._dec = np.radians(dec)
    b.compute(obs)
    return alt2airmass(b.alt)

def radec_obj_dist(obs, ra, dec, obj):
    b = ephem.FixedBody()
    b._ra = np.radians(ra)
    b._dec = np.radians(dec)
    b.compute(obs)
    obj.compute(obs)
    return np.degrees(ephem.separation(b, obj))

def radec_report(obs, ra, dec):
    am = radec2airmass(obs, ra, dec)
    ms = radec_obj_dist(obs, ra, dec, ephem.Moon())
    m = ephem.Moon()
    m.compute(obs)
    msg = (
        'Airmass: {:.2f}\n'
        'MoonSep: {:.1f} deg '
        '(moon is at {:.1f} deg, {:.1f} deg)'
    ).format(am, ms, np.degrees(m.ra), np.degrees(m.dec))
    return msg

In [5]:
import json

def gen_decam_json(ra, dec, filters, obj_name, fname, dither=True):
    exp_time_filters = {'g':30, 'r':30, 'i':20, 'z':20, 'Y':20 }
    
    exposures = []
    
    dither_scale = 0.262 * 4096 / 3600.
    d_ra = np.array([0.0, -0.5, 0.5]) * dither_scale / np.cos(np.radians(dec))
    d_dec = np.array([0.5, -0.5, -0.5]) * 0.5 * dither_scale
    
    if not dither:
        d_ra = [0.]
        d_dec = [0.]
    
    
    for da,dd in zip(d_ra, d_dec):
        for filt in filters:
            exposures.append({
                'expType': 'object',
                'object': 'DECaPS_{:s}_{:s}'.format(obj_name, filt),
                'expTime': exp_time_filters[filt],
                'filter': filt,
                'RA': np.mod(ra+da, 360.),
                'dec': dec+dd
            })

    f = open(fname, 'w')
    json.dump(exposures, f, indent=2, separators=(',',': '))
    f.close()

In [6]:
def proj_stereographic(theta, phi):
    '''
    Inputs:
      theta = polar angle
      phi = azimuthal angle
    Outputs:
      (x, y) = cartesian coordinates
    '''
    R = np.tan(0.5 * theta)
    return R * np.cos(phi), R * np.sin(phi)

In [7]:
decam = ephem.Observer()
decam.lon = '-70.806525'
decam.lat = '-30.169661'
decam.elev = 2207.0 # meters
decam.temp = 10.0 # deg celsius; average temp for August
decam.pressure = 780.0 # mbar
decam.horizon = -ephem.degrees('12:00:00.0')

# Calibrations we can do on August 1st between 10 and 12 twilight?

In [8]:
from astropy.coordinates import SkyCoord
obs = decam.copy()
obs.date='2018-08-01 22:56'

DES_SN={'C1':['03h37m05.83s -27d06m41.8s'],'C2':['03h37m05.83s -29d05m18.2s'],'C3':['03h30m35.62s -28d06m00.0s'], 'E1':['00h31m29.86s -43d00m34.6s'],
        'E2':['00h38m00.00s -43d59m52.8s'], 'S1':['02h51m16.80s +00d00m00.0s'],'S2':['02h44m46.66s -00d59m18.2s'],
        'X1':['02h17m54.17s -04d55m46.2s'],'X2':['02h22m39.48s -06d24m43.6s'],'X3':['02h25m48.00s -04d36m00.0s']}

for field in DES_SN:
    c=SkyCoord(DES_SN[field], frame='icrs')
    print(field)
    print("ra={:.2f},dec={:.2f}".format(c.ra.deg[0],c.dec.deg[0]))
    print radec_report(obs, c.ra.deg, c.dec.deg)
    print('\n')
    

X2
ra=35.66,dec=-6.41
Airmass: 11.86
MoonSep: 28.3 deg (moon is at 8.0 deg, -1.3 deg)


X3
ra=36.45,dec=-4.60
Airmass: 11.86
MoonSep: 28.8 deg (moon is at 8.0 deg, -1.3 deg)


S2
ra=41.19,dec=-0.99
Airmass: 11.86
MoonSep: 33.4 deg (moon is at 8.0 deg, -1.3 deg)


C3
ra=52.65,dec=-28.10
Airmass: 11.86
MoonSep: 50.5 deg (moon is at 8.0 deg, -1.3 deg)


C2
ra=54.27,dec=-29.09
Airmass: 11.86
MoonSep: 52.2 deg (moon is at 8.0 deg, -1.3 deg)


C1
ra=54.27,dec=-27.11
Airmass: 11.86
MoonSep: 51.4 deg (moon is at 8.0 deg, -1.3 deg)


X1
ra=34.48,dec=-4.93
Airmass: 11.86
MoonSep: 26.9 deg (moon is at 8.0 deg, -1.3 deg)


E1
ra=7.87,dec=-43.01
Airmass: 11.86
MoonSep: 41.6 deg (moon is at 8.0 deg, -1.3 deg)


S1
ra=42.82,dec=0.00
Airmass: 11.86
MoonSep: 35.1 deg (moon is at 8.0 deg, -1.3 deg)


E2
ra=9.50,dec=-44.00
Airmass: 11.86
MoonSep: 42.6 deg (moon is at 8.0 deg, -1.3 deg)




# Calibrations we can do if there is ten minutes leftover (at end of night) on August 1st?

In [9]:
from astropy.coordinates import SkyCoord
obs = decam.copy()
obs.date='2018-08-02 04:39'

DES_SN={'C1':['03h37m05.83s -27d06m41.8s'],'C2':['03h37m05.83s -29d05m18.2s'],'C3':['03h30m35.62s -28d06m00.0s'], 'E1':['00h31m29.86s -43d00m34.6s'],
        'E2':['00h38m00.00s -43d59m52.8s'], 'S1':['02h51m16.80s +00d00m00.0s'],'S2':['02h44m46.66s -00d59m18.2s'],
        'X1':['02h17m54.17s -04d55m46.2s'],'X2':['02h22m39.48s -06d24m43.6s'],'X3':['02h25m48.00s -04d36m00.0s']}

for field in DES_SN:
    c=SkyCoord(DES_SN[field], frame='icrs')
    print(field)
    print("ra={:.2f},dec={:.2f}".format(c.ra.deg[0],c.dec.deg[0]))
    print radec_report(obs, c.ra.deg, c.dec.deg)
    print('\n')

X2
ra=35.66,dec=-6.41
Airmass: 8.22
MoonSep: 25.6 deg (moon is at 11.0 deg, -0.3 deg)


X3
ra=36.45,dec=-4.60
Airmass: 10.24
MoonSep: 26.0 deg (moon is at 11.0 deg, -0.3 deg)


S2
ra=41.19,dec=-0.99
Airmass: 11.86
MoonSep: 30.4 deg (moon is at 11.0 deg, -0.3 deg)


C3
ra=52.65,dec=-28.10
Airmass: 11.86
MoonSep: 48.7 deg (moon is at 11.0 deg, -0.3 deg)


C2
ra=54.27,dec=-29.09
Airmass: 11.86
MoonSep: 50.4 deg (moon is at 11.0 deg, -0.3 deg)


C1
ra=54.27,dec=-27.11
Airmass: 11.86
MoonSep: 49.6 deg (moon is at 11.0 deg, -0.3 deg)


X1
ra=34.48,dec=-4.93
Airmass: 7.92
MoonSep: 24.1 deg (moon is at 11.0 deg, -0.3 deg)


E1
ra=7.87,dec=-43.01
Airmass: 1.48
MoonSep: 42.7 deg (moon is at 11.0 deg, -0.3 deg)


S1
ra=42.82,dec=0.00
Airmass: 11.86
MoonSep: 32.1 deg (moon is at 11.0 deg, -0.3 deg)


E2
ra=9.50,dec=-44.00
Airmass: 1.51
MoonSep: 43.7 deg (moon is at 11.0 deg, -0.3 deg)




# Example of generating the calibration jsons

In [10]:
gen_decam_json(7.87, -43.01, 'izY', 'C2', 'calib_izY_C2_Aug1_endofnight.json', dither=False)
gen_decam_json(7.87, -43.01, 'g', 'C2', 'calib_g_C2_Aug1_endofnight.json', dither=False)
gen_decam_json(7.87, -43.01, 'r', 'C2', 'calib_r_C2_Aug1_endofnight.json', dither=False)